In [1]:
from model import *
import csv

In [2]:
model = HousingMarket(rental_cost = 2000, initial_households=2000)
model.run_model(1)
data = model.datacollector.get_agent_vars_dataframe()

In [3]:
data["Income"]

Step  AgentID
1     21         1748.366030
      22         2347.880977
      23          541.442435
      24         2065.824859
      25         1544.193513
                    ...     
      2016       1799.759400
      2017       2200.732936
      2018       2370.625415
      2019       2504.139114
      2020       1376.694588
Name: Income, Length: 2000, dtype: float64

In [ ]:
data["Mean Income"].plot()

In [ ]:
model = HousingMarket(rental_cost = 10000)
model.run_model(1000)
data = model.datacollector.get_model_vars_dataframe()

In [ ]:
data["Gini"].plot()

In [ ]:
model = HousingMarket(rental_cost = 10)
model.run_model(1000)
data = model.datacollector.get_model_vars_dataframe()

In [ ]:
data["Gini"].plot()

In [ ]:
model = HousingMarket(rental_cost = 1000)
model.run_model()
data = model.datacollector.get_model_vars_dataframe()

In [ ]:
data["Gini"].plot()